# 本文件说明
- 读取数据、预处理数据

# 基本设置

In [1]:
import xlrd

%matplotlib inline
import numpy as np
import pandas as pd
from toolkits.nlp import pre_cor_circ
from toolkits.nlp import pre_cor_cbrc

import warnings
warnings.filterwarnings('ignore')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.038 seconds.
Prefix dict has been built succesfully.


## 一些函数

In [2]:
def predict_right(folder, file_list):
    '''预测正确数据'''
    right_data = pd.DataFrame()
    for file_name in file_list:
        currentPath  = '{0}/{1}'.format(folder, file_name)
        excel = xlrd.open_workbook(currentPath)
        sheet_names = [sheet.name for sheet in excel.sheets()]
        for sheet_name in sheet_names:    
            tmp_data = pd.read_excel(currentPath, sheet_name)
    #         tmp_data = tmp_data[tmp_data['R_W'] == 'Right']
            print('去空值前： ', tmp_data.shape, file_name, sheet_name)
#             tmp_data = tmp_data[tmp_data['备注'] != '删除']
    #         tmp_data = tmp_data.dropna(subset = ['raw_title'], axis = 0)
#             print('去空值后： ', tmp_data.shape, file_name, sheet_name)
            right_data = pd.concat([right_data, tmp_data], axis = 0)    
    return right_data

In [3]:
def correct_wrong_data(folder, file_list):
    '''预测错误修正后数据'''
    correct_wrong_data = pd.DataFrame()
    for file_name in file_list:
        currentPath  = '{0}/{1}'.format(folder, file_name)
        excel = xlrd.open_workbook(currentPath)
        sheet_names = [sheet.name for sheet in excel.sheets()]
        for sheet_name in sheet_names:    
            tmp_data = pd.read_excel(currentPath, sheet_name)
    #         tmp_data = tmp_data[tmp_data['R_W'] == 'Wrong']
            print('去删除前： ', tmp_data.shape, file_name, sheet_name)
            tmp_data = tmp_data[tmp_data['备注'] != '删除']
    #         tmp_data = tmp_data.dropna(subset = ['raw_title'], axis = 0)
            print('去删除后： ', tmp_data.shape, file_name, sheet_name)

            tmp_data['人工判断'] = tmp_data.apply(lambda x:1 if x['label'] == x['备注'] else 0, axis = 1)
            tmp_data['算法判断'] = tmp_data.apply(lambda x:1 if x['predict_label'] == x['备注'] else 0, axis = 1)
            print('人工误判率： %0.2f'%(1 - tmp_data['人工判断'].sum() / tmp_data.shape[0]),
                  '算法误判率： %0.2f'%(1 - tmp_data['算法判断'].sum() / tmp_data.shape[0]))
            print()

            correct_wrong_data = pd.concat([correct_wrong_data, tmp_data], axis = 0)
    return correct_wrong_data

In [4]:
def get_new_data(folder, file_list):
    '''新补充数据'''
    new_data = pd.DataFrame()
    for file_name in file_list:
        currentPath  = '{0}/{1}'.format(folder, file_name)
        excel = xlrd.open_workbook(currentPath)
        sheet_names = [sheet.name for sheet in excel.sheets()]
        for sheet_name in sheet_names:    
            tmp_data = pd.read_excel(currentPath, sheet_name)
            print(tmp_data.columns)
            tmp_data = tmp_data[['label', 'title', 'content']]
            print('去空值前： ', tmp_data.shape, file_name, sheet_name)
            tmp_data = tmp_data.dropna(subset = ['label'], axis = 0)
            print('去空值后： ', tmp_data.shape, file_name, sheet_name)
            new_data = pd.concat([new_data, tmp_data], axis = 0)    
    return new_data

In [5]:
def get_feedback_data(folder, file_list):
    '''模型预测后反馈数据'''
    feedback_data = pd.DataFrame()
    label_list = []
    sum_list = []
    right_list = []
    for file_name in file_list:
        currentPath  = '{0}/{1}'.format(folder, file_name)
        excel = xlrd.open_workbook(currentPath)
        sheet_names = [sheet.name for sheet in excel.sheets()]
        for sheet_name in sheet_names:   
            tmp_data = pd.read_excel(currentPath, sheet_name)    
            tmp_data = tmp_data[tmp_data['label'] != '删除']
            tmp_data_1 = tmp_data[tmp_data['label'] == sheet_name]

            label_list.append(sheet_name)
            sum_list.append(tmp_data.shape[0])
            right_list.append(tmp_data_1.shape[0])
            print('类别：', sheet_name, '总数：', tmp_data.shape[0], 
                  '正确数', tmp_data_1.shape[0], 
                  '正确比例', tmp_data_1.shape[0]/tmp_data.shape[0])

            tmp_data_1 = tmp_data[['label', 'title', 'content']]
            print('去空值前： ', tmp_data_1.shape, file_name, sheet_name)
            tmp_data_1 = tmp_data_1.dropna(subset = ['label'], axis = 0)
            print('去空值后： ', tmp_data_1.shape, file_name, sheet_name)
            feedback_data = pd.concat([feedback_data, tmp_data_1], axis = 0)
            print()    
            
    return feedback_data

In [6]:
def pre_save(pre_func, data, save_folder):
    print(data.shape)
    print('save_folder: ', save_folder)
    
    titles = pre_func(data['title'].tolist())
    print('title num: ', len(titles))
    save_filename = save_folder + 'corpus/titles.txt'
    fid = open(save_filename, "w+", encoding='UTF-8')
    for line in titles:
        fid.write(line + '\n')
    fid.close()  
    
#     print(len(data['content'].tolist()))
    contents = pre_func(data['content'].tolist())
    print('content num: ', len(contents))
    print(contents[0])
    # contents = [re.sub(r'[a-z]*', '', x) for x in contents]
    # print(len(contents))
    # print(contents[:2])
    coprus_save_filename = save_folder + 'corpus/contents.txt'
    f = open(coprus_save_filename, "w+", encoding='UTF-8')
    for line in contents:
        f.write(line + '\n')
    f.close()    

    label = data['label'].tolist()
    print('label num: ', len(label))
    coprus_save_filename = save_folder + 'corpus/labels.txt'
    f = open(coprus_save_filename, "w+", encoding='UTF-8')
    for line in label:
        f.write(str(line) + '\n')
    f.close()

    data.to_excel(save_folder + 'corpus/title_content_label.xlsx', index = False)
    

# 保险业：1 八分类

## 读取数据

In [7]:
folder = 'circ_data_class/20180910'

### 预测正确数据

In [8]:
file_list_1 = ['circ_class_sel_else_20180827.xlsx',]
circ_raw_class_1 = predict_right(folder, file_list_1)

circ_raw_class_1['label'] = circ_raw_class_1['label'].apply(lambda x:x.strip())    
circ_raw_class_1 = circ_raw_class_1[['label', 'raw_title', 'raw_content']]
circ_raw_class_1.columns = ['label', 'title', 'content']
print('总数：', circ_raw_class_1.shape)
circ_raw_class_1.head()

去空值前：  (1284, 8) circ_class_sel_else_20180827.xlsx 公司内部管理
去空值前：  (3197, 8) circ_class_sel_else_20180827.xlsx 噪音
去空值前：  (1320, 8) circ_class_sel_else_20180827.xlsx 行业
去空值前：  (1463, 8) circ_class_sel_else_20180827.xlsx 消费服务
去空值前：  (753, 8) circ_class_sel_else_20180827.xlsx 资本市场
去空值前：  (1511, 8) circ_class_sel_else_20180827.xlsx 监管
去空值前：  (500, 8) circ_class_sel_else_20180827.xlsx 产品销售
去空值前：  (1403, 8) circ_class_sel_else_20180827.xlsx 其他相关报道
总数： (11431, 3)


,label,title,content
0,公司内部管理,天安销售虚列费用为员工发福利 相关负责人被处罚,天安销售虚列费用为员工发福利 相关负责人被处罚\n\r2018.01.09 11:24:2...
1,公司内部管理,泰康人寿疑设招聘骗局：3500元底薪扣到剩1200元,泰康人寿疑设招聘骗局：3500元底薪扣到剩1200元\n\r 泰康人寿被爆疑设招聘“骗局...
2,公司内部管理,上海人寿整改应对监管 万能险收紧后仍过保费半数,上海人寿屡被监管关照 万能险收紧后仍占半壁江山\n\r(图片)\n\r投资时报\n\r01...
3,公司内部管理,“侨兴债”事件继续发酵，浙商财险9位高管被问责 保监会加大“双罚制”执行力度,“侨兴债”事件继续发酵，浙商财险9位高管被问责 保监会加大“双罚制”执行力度\n\r0\n...
4,公司内部管理,“开门红”时期监管严查销售误导 民生人寿连吃5罚单,“开门红”时期监管严查销售误导 民生人寿连吃5罚单\n\r2018.01.09 01:22...


In [9]:
circ_raw_class_1['label'].value_counts()

噪音        3197
监管        1511
消费服务      1463
其他相关报道    1403
行业        1320
公司内部管理    1284
资本市场       753
产品销售       500
Name: label, dtype: int64

### 预测错误修正后数据

In [10]:
file_list_2 = ['circ_class_sel_20180827-1.xlsx','circ_class_sel_20180827-2.xlsx']

circ_raw_class_2 = correct_wrong_data(folder, file_list_2)
circ_raw_class_2['label'] = circ_raw_class_2['label'].apply(lambda x:x.strip())  
circ_raw_class_2 = circ_raw_class_2[['备注', 'raw_title', 'raw_content']]
circ_raw_class_2.columns = ['label', 'title', 'content']
print('总数：', circ_raw_class_2.shape)
circ_raw_class_2.head()

去删除前：  (394, 8) circ_class_sel_20180827-1.xlsx 公司内部管理
去删除后：  (390, 8) circ_class_sel_20180827-1.xlsx 公司内部管理
人工误判率： 0.27 算法误判率： 0.47

去删除前：  (193, 8) circ_class_sel_20180827-1.xlsx 资本市场
去删除后：  (190, 8) circ_class_sel_20180827-1.xlsx 资本市场
人工误判率： 0.24 算法误判率： 0.48

去删除前：  (405, 8) circ_class_sel_20180827-1.xlsx 行业
去删除后：  (397, 8) circ_class_sel_20180827-1.xlsx 行业
人工误判率： 0.27 算法误判率： 0.53

去删除前：  (385, 8) circ_class_sel_20180827-1.xlsx 监管
去删除后：  (381, 8) circ_class_sel_20180827-1.xlsx 监管
人工误判率： 0.12 算法误判率： 0.75

去删除前：  (230, 8) circ_class_sel_20180827-2.xlsx 消费服务
去删除后：  (228, 8) circ_class_sel_20180827-2.xlsx 消费服务
人工误判率： 0.12 算法误判率： 0.71

去删除前：  (486, 8) circ_class_sel_20180827-2.xlsx 噪音
去删除后：  (469, 8) circ_class_sel_20180827-2.xlsx 噪音
人工误判率： 0.18 算法误判率： 0.65

去删除前：  (188, 8) circ_class_sel_20180827-2.xlsx 其他相关报道
去删除后：  (184, 8) circ_class_sel_20180827-2.xlsx 其他相关报道
人工误判率： 0.14 算法误判率： 0.65

去删除前：  (144, 8) circ_class_sel_20180827-2.xlsx 产品销售
去删除后：  (141, 8) circ_class_sel_20180827-2.xlsx 产品

,label,title,content
0,公司内部管理,太平人寿2016年原保费收入接近千亿大关,太平人寿2016年原保费收入接近千亿大关\n\r(图片) \n财经头条 1年前\n\r记者...
1,公司内部管理,经过一年半时间的筹建 首家科技保险公司正式开业,现在是科技时代，在我们日常生活中，处处都离不开现代科技，比如快捷付款之类。但最近保险科技最...
2,公司内部管理,安心财险“闲时退费”产品违反车险条款,近日，中国保监会对安心财险发布的监管函显示，安心财险“闲时退费”产品被定性为违反车险条款...
3,公司内部管理,上海人寿保险被保监会出具监管函 指其六方面违规,原标题：上海人寿保险被保监会出具监管函 指其六方面违规 千龙网北京10月11日讯 10月1...
4,公司内部管理,中国太平2018年一季度保费收入624.7亿元,4月13日，中国太平保险控股(股份代码：HK00966)公布2018年一季度保费收入。中国太...


In [11]:
circ_raw_class_2['label'].value_counts()

行业        491
公司内部管理    465
噪音        427
监管        254
产品销售      221
资本市场      215
其他相关报道    180
消费服务      127
Name: label, dtype: int64

### 新补充数据

In [12]:
file_list_3 = ['20180827_circ_class_1.xlsx'
               ] # 多个 sheet

circ_raw_class_3 = get_new_data(folder, file_list_3)

circ_raw_class_3['label'] = circ_raw_class_3['label'].apply(lambda x:x.strip())    
print('总数：', circ_raw_class_3.shape)
circ_raw_class_3.head()

Index(['title', 'content', 'label'], dtype='object')
去空值前：  (319, 3) 20180827_circ_class_1.xlsx 社会医疗保险
去空值后：  (319, 3) 20180827_circ_class_1.xlsx 社会医疗保险
总数： (319, 3)


,label,title,content
0,噪音,为什么医疗保险每年在加费用呢？相等的报销会增加吗？,为什么医疗保险每年在加费用呢？相等的报销会增加吗？ 待解决 匿名 问题与我的不同！ 我要立即...
1,噪音,临沂人注意了！有人看病多花好几万！只因不知道医疗保险的这一方面！,原标题:临沂人注意了！有人看病多花好几万！只因不知道医疗保险的这一方面！ 古语说得好 ...
2,噪音,如何计算医疗保险个人账户中每月可以进账多少钱？,除了公积金这个小金库，我们还有一个医疗保险每月可以进账，医疗保险中的个人账户可以用来看病买药...
3,噪音,医疗保险是否跟贫困补助有关联,医疗保险是否跟贫困补助有关联 待解决 匿名 问题与我的不同！ 我要立即提问 尊敬的用...
4,噪音,这事受理了|农村合作医疗保险哪些群体可以减免费用,原标题:这事受理了|农村合作医疗保险哪些群体可以减免费用 有拍友反映： 请问一下...


In [13]:
circ_raw_class_3['label'].value_counts()

噪音    319
Name: label, dtype: int64

### 模型预测后反馈数据

In [14]:
file_list_4 = ['20180904_circ_class_feedback.xlsx', '20180904_circ_class_feedback_1.xlsx',
              ] # 多个 sheet

circ_raw_class_4 = get_feedback_data(folder, file_list_4)

circ_raw_class_4['label'] = circ_raw_class_4['label'].apply(lambda x:x.strip())    
print('总数：', circ_raw_class_4.shape)
circ_raw_class_4.head()

类别： 噪音 总数： 200 正确数 187 正确比例 0.935
去空值前：  (200, 3) 20180904_circ_class_feedback.xlsx 噪音
去空值后：  (200, 3) 20180904_circ_class_feedback.xlsx 噪音

类别： 消费服务 总数： 198 正确数 137 正确比例 0.6919191919191919
去空值前：  (198, 3) 20180904_circ_class_feedback.xlsx 消费服务
去空值后：  (198, 3) 20180904_circ_class_feedback.xlsx 消费服务

类别： 产品销售 总数： 194 正确数 183 正确比例 0.9432989690721649
去空值前：  (194, 3) 20180904_circ_class_feedback.xlsx 产品销售
去空值后：  (194, 3) 20180904_circ_class_feedback.xlsx 产品销售

类别： 其他相关报道 总数： 200 正确数 155 正确比例 0.775
去空值前：  (200, 3) 20180904_circ_class_feedback.xlsx 其他相关报道
去空值后：  (200, 3) 20180904_circ_class_feedback.xlsx 其他相关报道

类别： 监管 总数： 198 正确数 128 正确比例 0.6464646464646465
去空值前：  (198, 3) 20180904_circ_class_feedback.xlsx 监管
去空值后：  (198, 3) 20180904_circ_class_feedback.xlsx 监管

类别： 行业 总数： 198 正确数 137 正确比例 0.6919191919191919
去空值前：  (198, 3) 20180904_circ_class_feedback.xlsx 行业
去空值后：  (198, 3) 20180904_circ_class_feedback.xlsx 行业

类别： 资本市场 总数： 199 正确数 171 正确比例 0.8592964824120602
去空值前：  (199, 3) 20180904_circ

,label,title,content
0,噪音,金塔县司法局三举措进一步加强对农民工追索劳动报酬提供法律援助服务,今年以来，金塔县司法局深刻认识新形势下做好对欠薪农民工提供法律服务和法律援助的重要性和紧...
1,噪音,经济日报：稳外贸需打“组合拳”,8月28日，江苏省太仓中集冷藏物流装备有限公司堆场上，各类冷藏箱整齐堆放，这些产品即将销...
2,行业,"捷越晨报│多家网贷平台宣布入股银行, 意在牌照与增信；深圳互金协会提示长租公寓“租金贷”风险。",29 星期三 2018年8月 1 宏观经济 宏观数据 上证指数 2777.98 -2.92 ...
3,消费服务,山东紧急调运２０台长臂挖掘机支援抢险救灾,新华社济南８月２８日电（记者叶婧）记者从山东省抗灾救灾与恢复重建指挥部了解到，应潍坊市请求，...
4,噪音,[军工]景嘉微：自主可控GPU稀缺标的，军民产业融合发展 国产替代系列之三,摘要 一、 军用图形显控龙头，自主可控GPU空间广阔 公司主要业务为军用图形显控产品，从最开...


In [15]:
circ_raw_class_4['label'].value_counts()

噪音        892
行业        420
公司内部管理    320
监管        249
产品销售      229
资本市场      216
其他相关报道    214
消费服务      195
Name: label, dtype: int64

## 合并数据

In [16]:
circ_raw_class = pd.concat([circ_raw_class_1, circ_raw_class_2,
                          circ_raw_class_3, circ_raw_class_4], 
                         axis = 0)

# circ_raw_class = cbrc_raw_class_4
circ_raw_class['title_content'] = circ_raw_class['title'].astype(str) + '。' + circ_raw_class['content'].astype(str)
circ_raw_class['label'] = circ_raw_class['label'].apply(lambda x:x.strip())
circ_raw_class['index'] = range(circ_raw_class.shape[0])
print('去重前： ', circ_raw_class.shape)
circ_raw_class = circ_raw_class.drop_duplicates(subset = 'title_content')
print('去重后： ', circ_raw_class.shape)
circ_raw_class.head()

去重前：  (16865, 5)
去重后：  (16865, 5)


,label,title,content,title_content,index
0,公司内部管理,天安销售虚列费用为员工发福利 相关负责人被处罚,天安销售虚列费用为员工发福利 相关负责人被处罚\n\r2018.01.09 11:24:2...,天安销售虚列费用为员工发福利 相关负责人被处罚。 天安销售虚列费用为员工发福利 相关负责人被...,0
1,公司内部管理,泰康人寿疑设招聘骗局：3500元底薪扣到剩1200元,泰康人寿疑设招聘骗局：3500元底薪扣到剩1200元\n\r 泰康人寿被爆疑设招聘“骗局...,泰康人寿疑设招聘骗局：3500元底薪扣到剩1200元。 泰康人寿疑设招聘骗局：3500元底薪...,1
2,公司内部管理,上海人寿整改应对监管 万能险收紧后仍过保费半数,上海人寿屡被监管关照 万能险收紧后仍占半壁江山\n\r(图片)\n\r投资时报\n\r01...,上海人寿整改应对监管 万能险收紧后仍过保费半数。 上海人寿屡被监管关照 万能险收紧后仍占半壁...,2
3,公司内部管理,“侨兴债”事件继续发酵，浙商财险9位高管被问责 保监会加大“双罚制”执行力度,“侨兴债”事件继续发酵，浙商财险9位高管被问责 保监会加大“双罚制”执行力度\n\r0\n...,“侨兴债”事件继续发酵，浙商财险9位高管被问责 保监会加大“双罚制”执行力度。 “侨兴债”事...,3
4,公司内部管理,“开门红”时期监管严查销售误导 民生人寿连吃5罚单,“开门红”时期监管严查销售误导 民生人寿连吃5罚单\n\r2018.01.09 01:22...,“开门红”时期监管严查销售误导 民生人寿连吃5罚单。 “开门红”时期监管严查销售误导 民生人...,4


In [17]:
print(circ_raw_class.shape)
circ_raw_class.groupby('label')['title'].count()

(16865, 5)


label
产品销售       950
公司内部管理    2069
其他相关报道    1797
噪音        4835
消费服务      1785
监管        2014
行业        2231
资本市场      1184
Name: title, dtype: int64

## 预处理

In [19]:
save_folder = 'circ_result_class/'
pre_save(pre_cor_circ.handle_contents, circ_raw_class, save_folder)

(16865, 5)
save_folder:  circ_result_class/
title num:  16865
content num:  16865
天安 销售 虚列费用 员工 发 福利 相关 负责人 被处罚 中国保监会 网站 当事人 张磊 身份证 号码 职务 天安 佰盈 销售 有限公司 宁波 分公司 简称 天安 销售 宁波 分公司 负责人 住所 宁波市 鄞州区 会展 路 号 层 D D 室 中华人民共和国 保险法 年 修正 我局 天安 销售 宁波 分公司 涉嫌 违法 一案 调查 审理 依法 当事人 告知 作出 行政处罚 事实 理由 当事人 依法 享有 权利 当事人 未 陈述 申辩 未 听证 经查 天安 销售 宁波 分公司 违法行为 财务 数据不真实 年 天安 销售 宁波 分公司 虚列 业务 招待费 万元 用于 员工 发放 福利 公司 负责人 张磊 负责 主管人员 变更 营业 场所 地址 未按规定 报告 年 月 天安 销售 宁波 分公司 营业 场所 地址 江北区 庄桥 街道 北环 西路 号 幢 二层 室 变更 鄞州区 会展 路 号 层 D D 室 年 月 日 事项 未按规定 我局 书面报告 公司 负责人 张磊 机构 负责 主管人员 上述事实 财务 凭证 租赁 合同 营业执照 情况 调查 笔录 任职 文件 现场 检查 事实 确认书 证据 证明 综上 我局 作出 处罚 天安 销售 宁波 分公司 财务 数据不真实 违反 中华人民共和国 保险法 年 修正 第八 十六条 第一百 三十二条 中华人民共和国 保险法 年 修正 一百七十 一条 给予 张磊 警告 并处 罚款 万元 行政处罚 天安 销售 宁波 分公司 变更 营业 场所 地址 未按规定 报告 违反 专业 代理 机构 监管 第十四条 专业 代理 机构 监管 七十条 给予 张磊 警告 处 罚款 万元 行政处罚 当事人 接到 处罚 决定书 之日起 日内持 缴款 码到 财政部 指定 家 代理 银行 中 任一 银行 同行 缴款 逾期 每日 罚款 数额 加处 罚款 缴款 码 行政处罚 决定书 送达 时 一并 告知 当事人 如对本 处罚 不服 接到 处罚 决定书 之日起 日内 依法 中国保监会 申请 行政复议 接到 处罚 决定书 之日起 月 内向 管辖权 人民法院 提起 行政诉讼 复议 诉讼 期间 停止 执行 宁波保监局

# 保险业：2 倾向性

## 读取数据

In [20]:
folder = 'circ_data_tendency/20180905'

### 预测正确数据

In [21]:
# file_list_1 = ['circ_class_sel_else_20180827.xlsx',]
# circ_raw_class_1 = predict_right(folder, file_list_1)

# circ_raw_class_1['label'] = circ_raw_class_1['label'].apply(lambda x:x.strip())    
# circ_raw_class_1 = circ_raw_class_1[['label', 'raw_title', 'raw_content']]
# circ_raw_class_1.columns = ['label', 'title', 'content']
# print('总数：', circ_raw_class_1.shape)
# circ_raw_class_1.head()

In [22]:
# circ_raw_class_1['label'].value_counts()

### 预测错误修正后数据

In [23]:
# file_list_2 = ['circ_class_sel_20180827-1.xlsx','circ_class_sel_20180827-2.xlsx']

# circ_raw_class_2 = correct_wrong_data(folder, file_list_2)
# circ_raw_class_2['label'] = circ_raw_class_2['label'].apply(lambda x:x.strip())  
# circ_raw_class_2 = circ_raw_class_2[['备注', 'raw_title', 'raw_content']]
# circ_raw_class_2.columns = ['label', 'title', 'content']
# print('总数：', circ_raw_class_2.shape)
# circ_raw_class_2.head()

In [24]:
# circ_raw_class_2['label'].value_counts()

### 新补充数据

In [25]:
file_list_3 = ['circ_0905_neg_raw_data.xlsx', 'circ_0905_pos_raw_data.xlsx',
               'circ_0905_pos_raw_data_add.xlsx'
               ] # 多个 sheet

circ_raw_tendency_3 = get_new_data(folder, file_list_3)

# circ_raw_tendency_3['label'] = circ_raw_tendency_3['label'].apply(lambda x:x.strip())    
print('总数：', circ_raw_tendency_3.shape)
circ_raw_tendency_3.head()

Index(['title', 'content', 'label'], dtype='object')
去空值前：  (368, 3) circ_0905_neg_raw_data.xlsx Sheet1
去空值后：  (368, 3) circ_0905_neg_raw_data.xlsx Sheet1
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (399, 3) circ_0905_pos_raw_data.xlsx Sheet1
去空值后：  (399, 3) circ_0905_pos_raw_data.xlsx Sheet1
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (255, 3) circ_0905_pos_raw_data_add.xlsx Sheet1
去空值后：  (255, 3) circ_0905_pos_raw_data_add.xlsx Sheet1
总数： (1022, 3)


,label,title,content
0,0,股份被转股东却不知情原保监会审查批复遭,股份被转股东却不知情 原保监会审查批复遭质疑大股东私自转让公司所有股权后，故意违法被工商部门...
1,0,产寿险分裂七大阵营一季度家险企亏损,产寿险分裂七大阵营：一季度72家险企亏损2017年，35家险企亏损过亿的消息尚未远去。201...
2,0,年险企车险承保亏损榜出炉家,2017年险企车险承保亏损榜出炉：49家共亏62亿 安盛天平成“亏损王”财险公司2017年“...
3,0,车险承保亏损榜第名国任财险连亏两年去,车险承保亏损榜第2名 国任财险连亏两年去年净利-2.7亿每经记者 胡 杨 每经编辑 王可然国...
4,0,去年保费缩两成退保超亿富德生命,"去年保费缩两成、退保超700亿，富德生命人寿一季度亏损延续专为<font color=""#F..."


In [26]:
circ_raw_tendency_3['label'].value_counts()

1    654
0    368
Name: label, dtype: int64

### 模型预测后反馈数据

In [27]:
# file_list_4 = ['20180904_circ_class_feedback.xlsx', '20180904_circ_class_feedback_1.xlsx',
#               ] # 多个 sheet

# circ_raw_class_4 = get_feedback_data(folder, file_list_4)

# circ_raw_class_4['label'] = circ_raw_class_4['label'].apply(lambda x:x.strip())    
# print('总数：', circ_raw_class_4.shape)
# circ_raw_class_4.head()

In [28]:
# circ_raw_class_4['label'].value_counts()

## 合并数据

In [29]:
# circ_raw_tendency = pd.concat([circ_raw_class_1, circ_raw_class_2,
#                           circ_raw_class_3, circ_raw_class_4], 
#                          axis = 0)

circ_raw_tendency = circ_raw_tendency_3
circ_raw_tendency['title_content'] = circ_raw_tendency['title'].astype(str) + '。' + circ_raw_tendency['content'].astype(str)
# circ_raw_tendency['label'] = circ_raw_tendency['label'].apply(lambda x:x.strip())
# circ_raw_tendency['index'] = range(circ_raw_tendency.shape[0])
print('去重前： ', circ_raw_tendency.shape)
circ_raw_tendency = circ_raw_tendency.drop_duplicates(subset = 'title_content')
print('去重后： ', circ_raw_tendency.shape)
circ_raw_tendency.head()

去重前：  (1022, 4)
去重后：  (767, 4)


,label,title,content,title_content
0,0,股份被转股东却不知情原保监会审查批复遭,股份被转股东却不知情 原保监会审查批复遭质疑大股东私自转让公司所有股权后，故意违法被工商部门...,股份被转股东却不知情原保监会审查批复遭。股份被转股东却不知情 原保监会审查批复遭质疑大股东私...
1,0,产寿险分裂七大阵营一季度家险企亏损,产寿险分裂七大阵营：一季度72家险企亏损2017年，35家险企亏损过亿的消息尚未远去。201...,产寿险分裂七大阵营一季度家险企亏损。产寿险分裂七大阵营：一季度72家险企亏损2017年，35...
2,0,年险企车险承保亏损榜出炉家,2017年险企车险承保亏损榜出炉：49家共亏62亿 安盛天平成“亏损王”财险公司2017年“...,年险企车险承保亏损榜出炉家。2017年险企车险承保亏损榜出炉：49家共亏62亿 安盛天平成“...
3,0,车险承保亏损榜第名国任财险连亏两年去,车险承保亏损榜第2名 国任财险连亏两年去年净利-2.7亿每经记者 胡 杨 每经编辑 王可然国...,车险承保亏损榜第名国任财险连亏两年去。车险承保亏损榜第2名 国任财险连亏两年去年净利-2.7...
4,0,去年保费缩两成退保超亿富德生命,"去年保费缩两成、退保超700亿，富德生命人寿一季度亏损延续专为<font color=""#F...",去年保费缩两成退保超亿富德生命。去年保费缩两成、退保超700亿，富德生命人寿一季度亏损延续专...


In [30]:
print(circ_raw_tendency.shape)
circ_raw_tendency.groupby('label')['title'].count()

(767, 4)


label
0    368
1    396
Name: title, dtype: int64

## 预处理

In [31]:
save_folder = 'circ_result_tendency/'
pre_save(pre_cor_circ.handle_contents, circ_raw_tendency, save_folder)

(767, 4)
save_folder:  circ_result_tendency/
title num:  767
content num:  767
股份 转 股东 不知情 原 保监会 审查 批复 遭 质疑 股东 私自 转让 公司 股权 违法 工商部门 吊销 营业执照 股东 申请 法院 强制 清算 发现 公司 早 年前 股份 转让 公司 并未 收到 转让款 申请 政府 信息 公开 部门 并未 法定期限 答复 股东 行政诉讼 邮件 提供 少部分 信息 涉及 股东 利益 大部分 信息 并未 提供 大连人 李岩 接受 獬豸 新闻 ID 记者 采访 时 谈起 此事 摇头 李岩 说 做梦 想到 身为 北京 毕保信 投资 有限公司 下称 毕保信 公司 股东 未有 知情 情况 持 股份 公司 非法 转让 毕保信 公司 未 收到 分文 股权 转让款 毕保信 公司 成立 年 月 日 工商部门 认缴 出资额 万元 股东 林建伟 出资 万元 持有 毕保信 公司 股权 股东 李岩 出资 万元 持有 股权 担任 公司 监事 期间 毕保信 公司 经营 年 月初 李岩 发现 毕保信 公司 营业执照 工商部门 予以 强制 吊销 同年 月 日 李岩 毕保信 公司 被吊销 营业执照 超过 法定期限 未 成立 清算组 清算 北京市 第三 中级 人民法院 下称 北京 三中 院 申请 毕保信 公司 强制 清算 法院 审理 查明 毕保信 公司 早 年 月 日 北京市工商局 以未 期限内 接受 年度 企业 年检 吊销 营业执照 责令 股东 清算组 清算 债权债务 办理 注销 登记 年 月 日 北京 三中 院 民事裁定 毕保信 公司 成立 强制 清算组 清算 过程 中 李岩 得知 早 年 月 日 毕保信 公司 安达 北美洲 FF 控股公司 下称 安达 北美洲 FF 华泰财产保险 股份 有限公司 下称 华泰 公司 签订 股份 转让 协议 协议 约定 安达 北美洲 受让 毕保信 公司 持有 华泰 公司 股份 每股 价格 美元 共计 美元 合计 人民币 元 对此 李岩 称 并未 股权 转让 事宜 做出 参加 股东会 决议 毕保信 公司 未 收到 股权 转让款 毕保信 公司 股东 股权 转让 致使 毕保信 公司 股份 对价 资产 丧失 侵害到 股东 毕保信 公司 享有 盈余 分配权 剩余 财产 索取权 股权 权能

# 保险业：3 预警

- 1: 预警
- 0: 非预警

## 读取数据

In [32]:
folder = 'circ_data_warn/20180621'

### 预测正确数据

In [33]:
# file_list_1 = ['circ_class_sel_else_20180827.xlsx',]
# circ_raw_class_1 = predict_right(folder, file_list_1)

# circ_raw_class_1['label'] = circ_raw_class_1['label'].apply(lambda x:x.strip())    
# circ_raw_class_1 = circ_raw_class_1[['label', 'raw_title', 'raw_content']]
# circ_raw_class_1.columns = ['label', 'title', 'content']
# print('总数：', circ_raw_class_1.shape)
# circ_raw_class_1.head()

In [34]:
# circ_raw_class_1['label'].value_counts()

### 预测错误修正后数据

In [35]:
# file_list_2 = ['circ_class_sel_20180827-1.xlsx','circ_class_sel_20180827-2.xlsx']

# circ_raw_class_2 = correct_wrong_data(folder, file_list_2)
# circ_raw_class_2['label'] = circ_raw_class_2['label'].apply(lambda x:x.strip())  
# circ_raw_class_2 = circ_raw_class_2[['备注', 'raw_title', 'raw_content']]
# circ_raw_class_2.columns = ['label', 'title', 'content']
# print('总数：', circ_raw_class_2.shape)
# circ_raw_class_2.head()

In [36]:
# circ_raw_class_2['label'].value_counts()

### 新补充数据

In [37]:
file_list_3 = ['20180621_circ_warn_data.xlsx'
               ] # 多个 sheet

circ_raw_warn_3 = get_new_data(folder, file_list_3)

# circ_raw_warn_3['label'] = circ_raw_warn_3['label'].apply(lambda x:x.strip())    
print('总数：', circ_raw_warn_3.shape)
circ_raw_warn_3.head()

Index(['title', 'content', 'label'], dtype='object')
去空值前：  (3551, 3) 20180621_circ_warn_data.xlsx Sheet1
去空值后：  (3551, 3) 20180621_circ_warn_data.xlsx Sheet1
总数： (3551, 3)


,label,title,content
0,1,防风险治乱象 逾900张罚单直指险企问题重灾区,今日报纸每经网首页 > 今日报纸 > ...
1,1,调结构回保障“保险+科技”已成行业新发力点,今日报纸每经网首页 > 今日报纸 > ...
2,1,补短板强监管 2017年保监会密集发文60余份,今日报纸每经网首页 > 今日报纸 > ...
3,1,寿险公司开门红产品调查： 上市险企增保障范围 中小险企暗提收益,本报记者 苏向杲 险企开门红首月已过大半，寿险公司推出开门红的产品也浮出水面。...
4,1,渎职行为违规行为终身追责,本报北京1月17日电 （记者曲哲涵）保监会17日发布《打赢保险业防范化解重大风险攻坚战...


In [38]:
circ_raw_warn_3['label'].value_counts()

0    1923
1    1628
Name: label, dtype: int64

### 模型预测后反馈数据

In [39]:
# file_list_4 = ['20180904_circ_class_feedback.xlsx', '20180904_circ_class_feedback_1.xlsx',
#               ] # 多个 sheet

# circ_raw_class_4 = get_feedback_data(folder, file_list_4)

# circ_raw_class_4['label'] = circ_raw_class_4['label'].apply(lambda x:x.strip())    
# print('总数：', circ_raw_class_4.shape)
# circ_raw_class_4.head()

In [40]:
# circ_raw_class_4['label'].value_counts()

## 合并数据

In [41]:
# circ_raw_class = pd.concat([circ_raw_class_1, circ_raw_class_2,
#                           circ_raw_class_3, circ_raw_class_4], 
#                          axis = 0)

circ_raw_warn = circ_raw_warn_3
circ_raw_warn['title_content'] = circ_raw_warn['title'].astype(str) + '。' + circ_raw_warn['content'].astype(str)
# circ_raw_warn['label'] = circ_raw_warn['label'].apply(lambda x:x.strip())
circ_raw_warn['index'] = range(circ_raw_warn.shape[0])
print('去重前： ', circ_raw_warn.shape)
circ_raw_warn = circ_raw_warn.drop_duplicates(subset = 'title_content')
print('去重后： ', circ_raw_warn.shape)
circ_raw_warn.head()

去重前：  (3551, 5)
去重后：  (3549, 5)


,label,title,content,title_content,index
0,1,防风险治乱象 逾900张罚单直指险企问题重灾区,今日报纸每经网首页 > 今日报纸 > ...,防风险治乱象 逾900张罚单直指险企问题重灾区。今日报纸每经网首页 > ...,0
1,1,调结构回保障“保险+科技”已成行业新发力点,今日报纸每经网首页 > 今日报纸 > ...,调结构回保障“保险+科技”已成行业新发力点。今日报纸每经网首页 > ...,1
2,1,补短板强监管 2017年保监会密集发文60余份,今日报纸每经网首页 > 今日报纸 > ...,补短板强监管 2017年保监会密集发文60余份。今日报纸每经网首页 > ...,2
3,1,寿险公司开门红产品调查： 上市险企增保障范围 中小险企暗提收益,本报记者 苏向杲 险企开门红首月已过大半，寿险公司推出开门红的产品也浮出水面。...,寿险公司开门红产品调查： 上市险企增保障范围 中小险企暗提收益。 本报记者 苏向杲 ...,3
4,1,渎职行为违规行为终身追责,本报北京1月17日电 （记者曲哲涵）保监会17日发布《打赢保险业防范化解重大风险攻坚战...,渎职行为违规行为终身追责。 本报北京1月17日电 （记者曲哲涵）保监会17日发布《打赢保...,4


In [42]:
print(circ_raw_warn.shape)
circ_raw_warn.groupby('label')['title'].count()

(3549, 5)


label
0    1921
1    1628
Name: title, dtype: int64

## 预处理

In [43]:
save_folder = 'circ_result_warn/'
pre_save(pre_cor_circ.handle_contents, circ_raw_warn, save_folder)

(3549, 5)
save_folder:  circ_result_warn/
title num:  3549
content num:  3549
今日 报纸 每经网 首页 今日 报纸 正文 防 风险 治乱 象 逾 张 罚单 直指 险企 重灾区 每日 经济 新闻 丨 每经 记者 胡 杨 每经 编辑 王可然 每经 记者 胡 杨 每经 编辑 王可然 回望 年 监管 姓监 四个 字 贯穿 始终 年 全国 保监系统 下发 张 监管函 超过 份 行政处罚 决定书 定期 通报 消费 投诉 情况 偿付能力 监管 工作 情况 首次 覆盖 全 行业 保险法 机构 公司 治理 现场 评估 体现 全方位 监管 之下 保险行业 乱象 整治 偿付能力 管理 为例 去年 三季度 末 家 公司 综合 偿付能力 溢额 万亿元 季度末 增加 亿元 人身险 公司 平均 核心偿付能力充足率 季度 百分点 连续 两个 季度 合计 处罚 金额 接近 亿据 每日 经济 新闻 记者 统计 年 全国 保监系统 一共 公开 多份 行政处罚 决定书 保监会 公布 余份 保监局 公布 多份 保监会 年 开出 张 罚单 相比 年 罚单 数量 增加 超过 五成 多份 处罚 决定书 涉及 家 保险机构 余位 关联 责任人 合计 处罚 金额 约 亿元 公司 性质 家 人身险 公司 家 财产险 公司 年内 吃 罚单 保险代理 公司 销售 公司 保险经纪 公司 相关 机构 接受 处罚 主力军 共有 家 机构 违规行为 监管部门 点名 往年 情况 销售误导 包括 电网 销 欺骗 投保人 虚列费用 给予 被保险人 保险合同 约定 利益 记录 客户 信息 不真实 聘任 不具有 任职 资格 人员 依旧 违规 重灾区 顽疾 深圳保监局 办公室 主任 汤志江 保监局 保监会 派出机构 违法行为 常规 工作 常抓不懈 进一步 监管 罚单 保监系统 日常 监管 体现 监管函 代表 保监会 违规 事件 面前 强 监管 态度 一年 中 保监会 发出 份 监管函 编号 推算 包含 未 披露 监管函 数十家 保险机构 被指 公司 治理 产品 业务 存在问题 每日 经济 新闻 记者 梳理 发现 年 保监会 开具 监管函 份 份 份 数量 皆 少于 年 公司 治理 评估 分 三批 下发 份 监管函 引发 行业 关注 去年 月 日 保监会 首发 张 监

# 银行业：1 八分类

## 读取数据

In [44]:
folder = 'cbrc_data_class/20180910'

### 预测正确数据

### 预测错误修正后数据

### 新补充数据

In [45]:
file_list_class_3 = ['20180814_cbrc_class_1.xlsx', '20180814_cbrc_class_2.xlsx',
               '20180817_cbrc_class_1.xlsx', '20180823_cbrc_class_1.xlsx',
               '20180823_cbrc_class_2.xlsx', '20180823_cbrc_class_3.xlsx', 
               '20180824_cbrc_class_1.xlsx', '20180824_cbrc_class_2.xlsx', 
               '20180824_cbrc_class_3.xlsx'] # 多个 sheet

cbrc_raw_class_3 = get_new_data(folder, file_list_class_3)

cbrc_raw_class_3['label'] = cbrc_raw_class_3['label'].apply(lambda x:x.strip())    
print('总数：', cbrc_raw_class_3.shape)
cbrc_raw_class_3.head()

Index(['url', 'title', 'content', 'label'], dtype='object')
去空值前：  (100, 3) 20180814_cbrc_class_1.xlsx 噪音
去空值后：  (100, 3) 20180814_cbrc_class_1.xlsx 噪音
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (100, 3) 20180814_cbrc_class_1.xlsx 公司内部管理
去空值后：  (100, 3) 20180814_cbrc_class_1.xlsx 公司内部管理
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (100, 3) 20180814_cbrc_class_1.xlsx 行业
去空值后：  (100, 3) 20180814_cbrc_class_1.xlsx 行业
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (100, 3) 20180814_cbrc_class_1.xlsx 监管
去空值后：  (100, 3) 20180814_cbrc_class_1.xlsx 监管
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (100, 3) 20180814_cbrc_class_1.xlsx 消费服务
去空值后：  (100, 3) 20180814_cbrc_class_1.xlsx 消费服务
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (107, 3) 20180814_cbrc_class_1.xlsx 资本市场
去空值后：  (107, 3) 20180814_cbrc_class_1.xlsx 资本市场
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (100, 3) 20180814_cbrc_class_1.xlsx 产品销售
去空值后： 

,label,title,content
0,噪音,小白学保险之基础篇——买好宝宝保险，第一步该做啥？,话说上回，在《给宝宝买保险，你说该信谁？》 中，身为一家之主的小白，想为家里挑些合适的保险，...
1,噪音,过诉讼时效的债务如何处理？看最高法院的5个答复(全),作者｜赫少华律师，远闻(上海)律师事务所，摘自微信公号“儒者如墨”，转载请注明 ▌一、...
2,噪音,国家旅游局局长李金早：推动从“高速旅游增长”向“优质旅游发展”转变,权威声音 1月8日全国旅游工作会议在厦门召开，国家旅游局局长李金早做了题为《以习近平新时代中...
3,噪音,"【法律宣传】状告百度们侵犯隐私,须用好公益诉讼利剑","1月1日,李书福在谈及电话、微信的保密问题时隔空diss马化腾,称其肯定天天在看我..."
4,噪音,百度被起诉，支付宝被约谈，网上隐私的边界在哪里？,据国家互联网信息办公室网站消息，针对近日新闻媒体报道的“支付宝年度账单事件”，1月6日，国家...


### 模型预测后反馈数据

In [46]:
file_list_class_4 = ['20180816_cbrc_class_feedback.xlsx', '20180816_cbrc_class_feedback_2.xlsx',
               '20180827_cbrc_class_feedback.xlsx'
              ] # 多个 sheet

cbrc_raw_class_4 = get_feedback_data(folder, file_list_class_4)

# cbrc_raw_class_4['label'] = cbrc_raw_class_4['label'].apply(lambda x:x.strip())    
print('总数：', cbrc_raw_class_4.shape)
cbrc_raw_class_4.head()

类别： 产品销售 总数： 453 正确数 25 正确比例 0.05518763796909492
去空值前：  (453, 3) 20180816_cbrc_class_feedback.xlsx 产品销售
去空值后：  (25, 3) 20180816_cbrc_class_feedback.xlsx 产品销售

类别： 噪音 总数： 700 正确数 700 正确比例 1.0
去空值前：  (700, 3) 20180816_cbrc_class_feedback.xlsx 噪音
去空值后：  (700, 3) 20180816_cbrc_class_feedback.xlsx 噪音

类别： 消费服务 总数： 290 正确数 3 正确比例 0.010344827586206896
去空值前：  (290, 3) 20180816_cbrc_class_feedback.xlsx 消费服务
去空值后：  (3, 3) 20180816_cbrc_class_feedback.xlsx 消费服务

类别： 行业 总数： 226 正确数 46 正确比例 0.20353982300884957
去空值前：  (226, 3) 20180816_cbrc_class_feedback.xlsx 行业
去空值后：  (46, 3) 20180816_cbrc_class_feedback.xlsx 行业

类别： 公司内部管理 总数： 120 正确数 7 正确比例 0.058333333333333334
去空值前：  (120, 3) 20180816_cbrc_class_feedback.xlsx 公司内部管理
去空值后：  (7, 3) 20180816_cbrc_class_feedback.xlsx 公司内部管理

类别： 资本市场 总数： 724 正确数 137 正确比例 0.18922651933701656
去空值前：  (724, 3) 20180816_cbrc_class_feedback_2.xlsx 资本市场
去空值后：  (137, 3) 20180816_cbrc_class_feedback_2.xlsx 资本市场

类别： 监管 总数： 258 正确数 78 正确比例 0.3023255813953488
去空值前：  (258, 3) 

,label,title,content
6,产品销售,弘扬中华优秀传统文化农行尊师系列信用卡精彩上市,近日，中国农业银行发布首张文化教育主题信用卡———尊师系列信用卡。该产品旨在弘扬尊师重道...
12,产品销售,信用卡额度怎么提高,\n\n平安的业务很多，而平安银行的主要业务那肯定就是信贷了，想必绝大多数小伙伴们想到的...
41,产品销售,有钱人都喜欢刷什么信用卡？,小编每次说“有钱人更容易申请到高端信用卡”的时候，总有不明真相的人跳出来怼我“有钱谁还用信用...
43,产品销售,今日工商银行理财产品排行榜（8月1日）,分享至：\n\n电邮\n\n打印\n\n评论(\n0\n)\n\n今日工商银行理财产品排行榜...
47,产品销售,信用卡是否可以提前还款？应该提前还款么？,在信用卡还款时，你会选择提前还款吗？怎样提前还款会对我们更有利？下面魔王就来说说提前还款那些...


## 合并数据

In [47]:
cbrc_raw_class = pd.concat([#cbrc_raw_class_4, cbrc_raw_class_4,
                          cbrc_raw_class_3, cbrc_raw_class_4], 
                         axis = 0)

# cbrc_raw = cbrc_raw_class_4
cbrc_raw_class['title_content'] = cbrc_raw_class['title'].astype(str) + '。' + cbrc_raw_class['content'].astype(str)
cbrc_raw_class['label'] = cbrc_raw_class['label'].apply(lambda x:x.strip())
# cbrc_raw_class['index'] = range(cbrc_raw_class.shape[0])
print('去重前： ', cbrc_raw_class.shape)
cbrc_raw_class = cbrc_raw_class.drop_duplicates(subset = 'title_content')
print('去重后： ', cbrc_raw_class.shape)
cbrc_raw_class.head()

去重前：  (8355, 4)
去重后：  (7920, 4)


,label,title,content,title_content
0,噪音,小白学保险之基础篇——买好宝宝保险，第一步该做啥？,话说上回，在《给宝宝买保险，你说该信谁？》 中，身为一家之主的小白，想为家里挑些合适的保险，...,小白学保险之基础篇——买好宝宝保险，第一步该做啥？。话说上回，在《给宝宝买保险，你说该信谁？...
1,噪音,过诉讼时效的债务如何处理？看最高法院的5个答复(全),作者｜赫少华律师，远闻(上海)律师事务所，摘自微信公号“儒者如墨”，转载请注明 ▌一、...,过诉讼时效的债务如何处理？看最高法院的5个答复(全)。 作者｜赫少华律师，远闻(上海)律...
2,噪音,国家旅游局局长李金早：推动从“高速旅游增长”向“优质旅游发展”转变,权威声音 1月8日全国旅游工作会议在厦门召开，国家旅游局局长李金早做了题为《以习近平新时代中...,国家旅游局局长李金早：推动从“高速旅游增长”向“优质旅游发展”转变。权威声音 1月8日全国旅...
3,噪音,"【法律宣传】状告百度们侵犯隐私,须用好公益诉讼利剑","1月1日,李书福在谈及电话、微信的保密问题时隔空diss马化腾,称其肯定天天在看我...","【法律宣传】状告百度们侵犯隐私,须用好公益诉讼利剑。 1月1日,李书福在谈及电话、微..."
4,噪音,百度被起诉，支付宝被约谈，网上隐私的边界在哪里？,据国家互联网信息办公室网站消息，针对近日新闻媒体报道的“支付宝年度账单事件”，1月6日，国家...,百度被起诉，支付宝被约谈，网上隐私的边界在哪里？。据国家互联网信息办公室网站消息，针对近日新...


In [48]:
print(cbrc_raw_class.shape)
cbrc_raw_class.groupby('label')['title'].count()

(7920, 4)


label
产品销售       807
公司内部管理     896
其他相关报道    1365
噪音         800
消费服务       877
监管        1334
行业         984
资本市场       857
Name: title, dtype: int64

## 预处理

In [49]:
save_folder = 'cbrc_result_class/'
pre_save(pre_cor_cbrc.handle_contents, cbrc_raw_class, save_folder)

(7920, 4)
save_folder:  cbrc_result_class/
title num:  7920
content num:  7920
话 说 回 宝宝 买保险 说 信 中 身为 一家之主 小白 想 家里 挑些 合适 产品 术语 绕晕 貌似 孩子 买保险 妈妈 上心 一筹莫展 之际 小白 偶然 结识 精算师 徐 老师 徐 老师 短暂 交流 小白 受益匪浅 内心 窃喜 终于 找到 谱 小白 迫不及待 徐 老师 发了 几份 计划书 希望 徐 老师 帮忙 分析 分析 徐 老师 计划书 心中 暗暗 叫苦 最最 基础 知识 懂 产品 这是 没上 产前 训练班 宝宝 徐 老师 回答 小白 眉头 紧 缓缓 问道 在你看来 小白 脱口而出 理财 工具 万一出 事儿 保险金 减轻负担 平安无事 当存 银行 徐 老师 哭笑不得 妈妈 可不是 这么回事 揭开 神秘 面纱 保障 未来 特定 事故 发生 时 给予 被保险人 受益人 经济 补偿 这是 教科书 定义 徐 老师 顾名思义 保障 风险 非 保 风险 风险 发生 带来 损失 带来 收益 炒股 炒房 赌博 买 彩票 不保 财产 意外 死亡 医疗 保 抗 风险 找 爱车 预备 胎 事故 说不定 发生 换上 出发 家庭 经济 预备 胎 风险 发生 至少 发生 不幸 宝宝 奶粉 钱 不用 担心 家庭 经济 垮掉 听 完徐 老师 讲解 小白 若有所思 生 疑问 为啥 接触 徐 老师 所说 徐 老师 手机 屏 叹 一口气 回复 小白 这口 锅 甩 代理人 代理人 现状 代理人 通病 不适 自动 省略 段 培训 时间 短 培训 质量 差 相关 保险代理 业务人员 上岗 前 接受 培训 时间 少于 小时 上岗 每人每年 接受 培训 教育 时间 累计 少于 小时 法律 知识 培训 职业道德 教育 少于 小时 小时 学习 经历 不足以 领域 入门级 菜鸟 高端 级 大师 小时 中 时间 极有 用于 销售 培训 夸夸其谈 谈笑风生 生活虎 狐假虎威 威逼利诱 诱 买保险 徐 老师 说 代理人 专业 外 服务 专业 服务差 扬长避短 销售 客观 公司 投资 类 兼顾 保障 投资收益 双重 需求 建议您 攻防 兼顾 角度 思考 投资 理财 架构 适度 防守 点上 加大 投入 投资 类 带给 投资收益 增加 保险金额 扩充 保障 利益 这话

# 银行业：2 倾向性

## 读取数据

In [50]:
folder = 'cbrc_data_tendency/20180622'

### 预测正确数据

In [51]:
# file_list_1 = ['circ_class_sel_else_20180827.xlsx',]
# circ_raw_class_1 = predict_right(folder, file_list_1)

# circ_raw_class_1['label'] = circ_raw_class_1['label'].apply(lambda x:x.strip())    
# circ_raw_class_1 = circ_raw_class_1[['label', 'raw_title', 'raw_content']]
# circ_raw_class_1.columns = ['label', 'title', 'content']
# print('总数：', circ_raw_class_1.shape)
# circ_raw_class_1.head()

In [52]:
# circ_raw_class_1['label'].value_counts()

### 预测错误修正后数据

In [53]:
# file_list_2 = ['circ_class_sel_20180827-1.xlsx','circ_class_sel_20180827-2.xlsx']

# circ_raw_class_2 = correct_wrong_data(folder, file_list_2)
# circ_raw_class_2['label'] = circ_raw_class_2['label'].apply(lambda x:x.strip())  
# circ_raw_class_2 = circ_raw_class_2[['备注', 'raw_title', 'raw_content']]
# circ_raw_class_2.columns = ['label', 'title', 'content']
# print('总数：', circ_raw_class_2.shape)
# circ_raw_class_2.head()

In [54]:
# circ_raw_class_2['label'].value_counts()

### 新补充数据

In [55]:
file_list_3 = ['cbrc_0609_pos_raw_data.xlsx', 'cbrc_0609_neg_raw_data.xlsx',
               ] # 多个 sheet

cbrc_raw_tendency_3 = get_new_data(folder, file_list_3)

# circ_raw_tendency_3['label'] = circ_raw_tendency_3['label'].apply(lambda x:x.strip())    
print('总数：', cbrc_raw_tendency_3.shape)
cbrc_raw_tendency_3.head()

Index(['content', 'label', 'title'], dtype='object')
去空值前：  (500, 3) cbrc_0609_pos_raw_data.xlsx Sheet1
去空值后：  (500, 3) cbrc_0609_pos_raw_data.xlsx Sheet1
Index(['content', 'label', 'title'], dtype='object')
去空值前：  (398, 3) cbrc_0609_neg_raw_data.xlsx Sheet1
去空值后：  (398, 3) cbrc_0609_neg_raw_data.xlsx Sheet1
总数： (898, 3)


,label,title,content
0,1,NaN,【消息面解析】 周三(12月14日)原油盘初，油价承压，美油跌逾1%。美油交投于52.3...
1,1,NaN,贵金属各大交易平台:在这两个转换下，总量数据在2010年达到高点后便持续处于下行通道当中，周...
2,1,NaN,关注：掌上有色：隔夜财经数据评论：非农利好提前消化美元持稳 中国数据向好金属表现较好期铜领...
3,1,NaN,周一就被行情怼死，有些意外直接破高，因为要上非农就可以直接涨，但非农冲高回落日线收阴，按理周...
4,1,NaN,今年以来，区环保局围绕区委区政府提出的“干好一三五”总体思路，根据“三年活动”“五大硬仗”总...


In [56]:
cbrc_raw_tendency_3['label'].value_counts()

1    500
0    398
Name: label, dtype: int64

### 模型预测后反馈数据

In [57]:
# file_list_4 = ['20180904_circ_class_feedback.xlsx', '20180904_circ_class_feedback_1.xlsx',
#               ] # 多个 sheet

# circ_raw_class_4 = get_feedback_data(folder, file_list_4)

# circ_raw_class_4['label'] = circ_raw_class_4['label'].apply(lambda x:x.strip())    
# print('总数：', circ_raw_class_4.shape)
# circ_raw_class_4.head()

In [58]:
# circ_raw_class_4['label'].value_counts()

## 合并数据

In [59]:
# circ_raw_tendency = pd.concat([circ_raw_class_1, circ_raw_class_2,
#                           circ_raw_class_3, circ_raw_class_4], 
#                          axis = 0)

cbrc_raw_tendency = cbrc_raw_tendency_3
cbrc_raw_tendency['title_content'] = cbrc_raw_tendency['title'].astype(str) + '。' + cbrc_raw_tendency['content'].astype(str)
# circ_raw_tendency['label'] = circ_raw_tendency['label'].apply(lambda x:x.strip())
# circ_raw_tendency['index'] = range(circ_raw_tendency.shape[0])
print('去重前： ', cbrc_raw_tendency.shape)
cbrc_raw_tendency = cbrc_raw_tendency.drop_duplicates(subset = 'title_content')
print('去重后： ', cbrc_raw_tendency.shape)
cbrc_raw_tendency.head()

去重前：  (898, 4)
去重后：  (896, 4)


,label,title,content,title_content
0,1,NaN,【消息面解析】 周三(12月14日)原油盘初，油价承压，美油跌逾1%。美油交投于52.3...,nan。【消息面解析】 周三(12月14日)原油盘初，油价承压，美油跌逾1%。美油交投于...
1,1,NaN,贵金属各大交易平台:在这两个转换下，总量数据在2010年达到高点后便持续处于下行通道当中，周...,nan。贵金属各大交易平台:在这两个转换下，总量数据在2010年达到高点后便持续处于下行通道...
2,1,NaN,关注：掌上有色：隔夜财经数据评论：非农利好提前消化美元持稳 中国数据向好金属表现较好期铜领...,nan。关注：掌上有色：隔夜财经数据评论：非农利好提前消化美元持稳 中国数据向好金属表现较...
3,1,NaN,周一就被行情怼死，有些意外直接破高，因为要上非农就可以直接涨，但非农冲高回落日线收阴，按理周...,nan。周一就被行情怼死，有些意外直接破高，因为要上非农就可以直接涨，但非农冲高回落日线收阴...
4,1,NaN,今年以来，区环保局围绕区委区政府提出的“干好一三五”总体思路，根据“三年活动”“五大硬仗”总...,nan。今年以来，区环保局围绕区委区政府提出的“干好一三五”总体思路，根据“三年活动”“五大...


In [60]:
print(cbrc_raw_tendency.shape)
cbrc_raw_tendency.groupby('label')['title_content'].count()

(896, 4)


label
0    398
1    498
Name: title_content, dtype: int64

## 预处理

In [61]:
save_folder = 'cbrc_result_tendency/'
pre_save(pre_cor_cbrc.handle_contents, cbrc_raw_tendency, save_folder)

(896, 4)
save_folder:  cbrc_result_tendency/
title num:  896
content num:  896
消息面 解析 周三 月 日 原油 盘初 油价 承压 美油 跌逾 美油 交投 美元 桶 跌幅 约 布油 交投 美元 桶 跌幅 约 OPEC 与非 OPEC 达成 减产 协议 提振 市场 信心 令 油价 近期 高位 国际 能源 署 IEA 最新 公布 月 报 显示 月 全球 原油 供应 创新 高 美国 石油 协会 API 公布 数据 显示 美国 上周 API 原油 库存 汽油 库存 录得 意外 大增 令 油价 承压 日内 公布 OPEC 月 报 美国 EIA 库存 报告 油价 后市 走向 提供 指引 周三 月 日 亚市 盘中 美元 指数 震荡 交投 关口 非美 货币 盘中 涨跌 现货 黄金 盘中 温和 反弹 美国 原油 盘中 小幅 下跌 日 内需 关注 英国 失业率 美国 PPI 零售 销售 工业 产出 EIA 原油 库存 数据 OPEC 公布 月度 原油 市场 报告 英国 央行 行长 卡尼 发表 讲话 北京 时间 周四 凌晨 点将 公布 美联储 决议 技术 面 分析 现货 原油 沥青 周初 冲高回 落后 走 技术 面 调整 收取 带有 一根 影响 阳线 走势 偏 震荡 行情 早间 跳空 低开 配合 预期 技术 面 回调 回 补缺 口 修复 均线 指标 日线图 K 线 回归 于布林 通道 内部 日内 下方 关注 日 均线 支撑 日 均线 高开 缺口 回补 位置 MACD 金叉 运行 红色 动能 缩减 迹象 RSI KDJ 黏合 勾头 向下 发散 指标 呈 弱势 操作 反弹 做空 小时 布 林带 缩口 K 线 运行 中轨 上方 受 均线 压制 下方 短线 关注 中轨 支撑 MACD 死叉 运行 短线 偏 空头 趋势 操作 建议 高 做空 现货 沥青 操作 建议 策略 做空 止损 目标 策略 跌破 反弹 做空 止损 目标 策略 首次 触及 做 止损 目标 策略 由木易 点金 撰稿 分析 以宁贵 沥青 点位 为准 仅供参考 附 国际原油 操作 建议 做空 止损 目标 跌破 反弹 做空 目标 现货 黄金 走势 弱势 修正 想 反弹 日线 均线 压制 力度 一个月 金价 遭遇 上方 均线 压制 呈 弱势 下行 趋势 周一 低

# 保存本文件

In [ ]:
if 0:
    import datetime as dt
    
    def output_HTML(read_file, output_file):
        from nbconvert import HTMLExporter
        import codecs
        import nbformat
        exporter = HTMLExporter()
        # read_file is '.ipynb', output_file is '.html'
        output_notebook = nbformat.read(read_file, as_version=4)
        output, resources = exporter.from_notebook_node(output_notebook)
        codecs.open(output_file, 'w', encoding='utf-8').write(output)

    html_file_folder = 'html_files'
    if not os.path.exists(html_file_folder):
        os.makedirs(html_file_folder)

    today = dt.datetime.now().strftime('%Y%m%d')
    current_file = 'circ_cor_model_2_train.ipynb'
    output_file = 'html_files\%s_%s.html'%(os.path.splitext(current_file)[0], today)
    output_HTML(current_file, output_file)